In [ ]:
# default_exp tabular.core

# tabular.core

> API details.

In [ ]:
#hide
#export
import pandas as pd
from fastai.data.external import *
from fastcore.all import *
from pathlib import PosixPath
from fastcore.test import *
from fastai.tabular.all import *
import fastai
from fastai.tabular.core import _maybe_expand
from itertools import chain
from sklearn.model_selection import train_test_split    

In [ ]:
#hide
from nbdev.showdoc import *

## Helpers to read files.

In [ ]:
#hide
#export
def str_to_path(file: str):
    "Convers a string to a Posixpath."
    if isinstance(file, str) and "~" in file:
        file = os.path.expanduser(file)

    file = Path(file)

    return file

In [ ]:
#hide
test_eq_type(Path(""), str_to_path(""))
test_eq_type(Path(""), str_to_path(Path("")))

In [ ]:
#export
def read_hdf(file:PosixPath, key: str = "/powerdata", key_metadata=None):
    "Reads a hdf5 table based on the given key."
    file = str_to_path(file)
    if "/" not in key: key = "/" + key
    with pd.HDFStore(file, "r") as store:
        if key in store.keys():
            df = store[key]
            if key_metadata is not None:
                df_meta = store[key_metadata]
                for c in df_meta: df[c] = df_meta[c].values[0]
        else:
            df = pd.DataFrame()
    return df

In [ ]:
#hide
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]},
                  index=['a', 'b', 'c'])
df.to_hdf('data.h5', key='df', mode='w')
test_eq(df, read_hdf("data.h5", key="df"))

In [ ]:
#export
def read_csv(file:PosixPath, sep:str =";"):
    "Reads a csv file."
    file = str_to_path(file)
    df = pd.read_csv(str(file), sep=sep)
    df.drop(["Unnamed: 0"], inplace=True, axis=1, errors="ignore")
    return df

In [ ]:
#hide
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]},)
df.to_csv('data.csv', sep=";")
test_eq(df, read_csv("data.csv", sep=";"))

In [ ]:
#export
def read_files(
    files:PosixPath,
    key:str ="/powerdata",
    key_metadata=None,
    sep:str=";",
    add_task_id=True
) -> pd.DataFrame:
    "Reads a number of CSV or HDF5 files depending on file ending."

    files = listify(files)
    dfs=L()
    for task_id,file in enumerate(files):
        if isinstance(file, str):
            file = str_to_path(file)

        if file.suffix == ".h5":
            df = read_hdf(file, key, key_metadata=key_metadata)
        elif file.suffix == ".csv":
            df = read_csv(file, sep=";")
        else:
            raise f"File ending of file {file} not supported."
        if add_task_id:df["TaskID"]=task_id
        dfs += df

    return dfs

In [ ]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]},
                  index=['a', 'b', 'c'])
df.to_hdf('data.h5', key='df', mode='w')
test_eq(df, read_files("data.h5", key="df", add_task_id=False)[0])

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]},)
df.to_csv('data.csv', sep=";")
test_eq(df, read_files("data.csv", add_task_id=False)[0])

## Feature Engineering

In [ ]:
#export
# this is merely a class to differentiate between fastai processing and renewbale pre-processing functionality
class RenewablesTabularProc(TabularProc):
    include_in_new=False
    pass

In [ ]:
class TestProc(RenewablesTabularProc): pass
test_eq(isinstance(TestProc(), RenewablesTabularProc), True)

In [ ]:
#export
class CreateTimeStampIndex(RenewablesTabularProc):
    order=0
    include_in_new=True
    def __init__(self, col_name, offset_correction=None):
        self.col_name = col_name
        self.offset_correction = offset_correction

    def encodes(self, to):
        df = to.items
        
        def create_timestamp_index(df, drop_index=True):
            df.reset_index(drop=drop_index, inplace=True)
            df.rename({self.col_name: "TimeUTC"}, axis=1, inplace=True)
            #  in case the timestamp is index give it a proper timestamp,e.g., in GermanSolarFarm dataset
            if "0000-" in str(df.TimeUTC[0]):
                df.TimeUTC = df.TimeUTC.apply(
                    lambda x: x.replace("0000-", "2015-").replace("0001-", "2016-")
                )
            df.TimeUTC = pd.to_datetime(df.TimeUTC, infer_datetime_format=True, utc=True)
            df.set_index("TimeUTC", inplace=True)
            df.index = df.index.rename("TimeUTC")

            #  for GermanSolarFarm, the index is not corret. Should have a three hour resolution but is one...
            if self.offset_correction is not None:
                i, new_index = 0, []
                for cur_index in df.index:
                    new_index.append(cur_index + pd.DateOffset(hours=i))
                    i += self.offset_correction
                df.index = new_index

        if self.col_name in df.columns:
            create_timestamp_index(df, drop_index=True)
        # properly already processed
        elif self.col_name == to.items.index.name:  
            create_timestamp_index(df, drop_index=False)
        else:
            warnings.warn(f"Timetamps column {self.col_name} not in columns {df.columns} or df.index.name")

In [ ]:
def get_test_data(index=None):
    df = pd.DataFrame(index=range(0,5), columns = ['A', 'B', 'C'] ).fillna(0)
    if index is not None: df["TimeStamps"] = index
    return TabularPandas(df)
# tests basic functionality to set a proper timestamp based index
index = ['2015-01-01-01', '2015-01-01-02', '2015-01-02-03', '2015-02-01-23', '2015-02-01-13'] 
to = get_test_data(index)
test_eq(CreateTimeStampIndex(col_name="TimeStamps")(to).items.index, pd.to_datetime(index, utc=True))

# corrects missing year
index_missing_year = ['0000-01-01-01', '0000-01-01-02', '0000-01-02-03', '0000-02-01-23', '0000-02-01-13'] 
to = get_test_data(index_missing_year)
test_eq(CreateTimeStampIndex(col_name="TimeStamps")(to).items.index, pd.to_datetime(index, utc=True))

# check if warning is triggered, due to wrong column name
to = get_test_data(index)
test_call = lambda: CreateTimeStampIndex(col_name="FalseColumnName")(to)
test_warns(test_call)

In [ ]:
#export
def get_samples_per_day(df, n_samples_to_check=100, expected_samples=[8,24,96]):
    """
    Extract the amount of entries per day from the DataFrame in the first n_samples_to_check.
    Aborts, ones the first meaningful *number of samples per day* is found.
    Parameters
    ----------
    df : pandas.DataFrame
        the DataFrame used for the conversion.

    Returns
    -------
    integer
        amount of entries per day.
    """
    samples_per_day = -1
    
    if len(df) == 0: return samples_per_day
    mins = 0
    for i in range(1, min(n_samples_to_check,len(df))):
        mins = (df.index[i] - df.index[i -1]).seconds // 60
        if (24*60)%mins==0:
            samples_per_day = (24*60)/mins
            if samples_per_day in expected_samples: break

    if samples_per_day == -1:
        raise ValueError(f"{mins} is an unknown sampling time.")
        
    return int(samples_per_day)



In [ ]:
def test_data_samples_per_day(index):
    return pd.DataFrame(index=pd.to_datetime(index),
                      columns = ['A', 'B', 'C'] ).fillna(0)
df = test_data_samples_per_day(index = pd.to_datetime(['2018-01-01-00:00', '2019-01-01-04:00', '2020-01-01-07:00',] ))
test_eq(8, get_samples_per_day(df))
df = test_data_samples_per_day(index = pd.to_datetime(['2018-01-01-00:00', '2019-01-01-02:00', '2020-01-01-03:00',] ))
test_eq(24, get_samples_per_day(df))
df = test_data_samples_per_day(index = pd.to_datetime(['2018-01-01-00:00', '2019-01-01-01:15', '2020-01-01-01:30',] ))
test_eq(96, get_samples_per_day(df))
test_eq(-1, get_samples_per_day(test_data_samples_per_day([])))

In [ ]:
#export
def _interpolate_df(df, sample_time="15Min", limit=5, drop_na=False):
        df = df[~df.index.duplicated()]
        upsampled = df.resample(sample_time)
        df  = upsampled.interpolate(method="linear", limit=limit)
        
        if drop_na: df = df.dropna(axis=0)

        if "Hour" in df.columns:
            df["Hour"] = df.index.hour
        if "Month" in df.columns:
            df["Month"] = df.index.month
        if "Day" in df.columns:
            df["Day"] = df.index.day
        if "Week" in df.columns:
            df["Week"] = df.index.week

        return df

In [ ]:
def test_data_interpolate(index=pd.to_datetime(['2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-03:00'])):
    np.random.seed(2)
    df = pd.DataFrame(index=pd.to_datetime(index),
                      data=np.random.randint(0,10,size=(len(index),3)),
                  columns = ['A', 'B', 'C'] )
    return df


In [ ]:
df = test_data_interpolate()
inp_df = _interpolate_df(df, limit=5, drop_na=True)
# instead of three values per hours we now have four plus the last timesamp
# duplicated values are droppped
test_eq(9, inp_df.shape[0])
test_eq(2, inp_df.iloc[-2,0])
# one for every timestamp plus one for the first two timestamps, assure that duplicates are dropped
df = test_data_interpolate(index = pd.to_datetime(['2018-01-01-01:00', '2018-01-01-01:00', '2018-01-01-03:00', '2018-01-01-04:00']),)
test_eq(5, _interpolate_df(df, limit=1, drop_na=True).shape[0])

In [ ]:
#export
def _apply_group_by(to:pd.DataFrame, group_by_col, func, **kwargs):
    if group_by_col in to.columns:
        dfs = L()
        for k,df_g in to.groupby(group_by_col):
            dfs += func(df_g, **kwargs)
        df = pd.concat(dfs, axis=0)
    else:
        df = func(to, **kwargs)
    return df

In [ ]:
#export
class Interpolate(RenewablesTabularProc):
    order=50
    include_in_new=True
    def __init__(self, sample_time = "15Min", limit=5, drop_na=True, group_by_col="TaskID"):
        self.sample_time = sample_time
        self.limit = limit
        self.drop_na = drop_na
        self.group_by_col = group_by_col
    
    def setups(self, to: Tabular):
        self.n_samples_per_day = get_samples_per_day(to.items)
        if self.n_samples_per_day == -1:
            warnings.warn("Could not determine samples per day. Skip processing.")
    
    def encodes(self, to):
        
        if self.n_samples_per_day == -1: return
        
        # if values of a columns are the same in each row (categorical features)
        # we make that those stay the same during interpolation
        if self.group_by_col in to.items.columns:
            d = defaultdict(object)
            non_unique_columns = L()
            for group_id, df in to.items.groupby(self.group_by_col):
                for c in df.columns:
                    if len(df[c].unique())==1 and c!=self.group_by_col:
                        d[(group_id,c)] = df[c][0]
                    else:
                        non_unique_columns += c
            
            non_unique_columns = np.unique(non_unique_columns)
        else:
            non_unique_columns = to.items.columns
        # interpolate non unique columns         
        df = _apply_group_by(to.items.loc[:,np.unique(non_unique_columns)], self.group_by_col, _interpolate_df)
        to.items = df
        if self.group_by_col in to.items.columns:
            for group_id,col_name in d.keys():
                mask = to[self.group_by_col]==group_id
                to.items.loc[mask,col_name]=d[(group_id, col_name)] 
        
        # TODO: to infer dtype through pd.inferdtype
        # use this for conversion
        # in case there is an object inside, throw an error
        if len(to.cont_names)>0:
            mask = to[to.cont_names].isna().values[:,0]
            to.items = to.items[~mask]
        # pandas converts the datatype to float if np.NaN is present, lets revert that                
        to.items = to.items.convert_dtypes()

In [ ]:
df = test_data_interpolate(index = pd.to_datetime(['2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-03:00']),)
df["TaskID"] = [1,1,2,2,2]
to = TabularPandas(df, procs=Interpolate, do_setup=True)
test_eq(5, to.items.loc[to.items.TaskID==1].shape[0])
test_eq(9, to.items.loc[to.items.TaskID==2].shape[0])

df = test_data_interpolate(index = pd.to_datetime(['2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-03:00']),)
df["TaskID"] = [1,1,2,2,2]
df["B"] = [1,1,2,2,2]
to = TabularPandas(df, cont_names="A", procs=Interpolate, do_setup=True)
test_eq([1,1,1,1,1,2,2,2,2,2,2,2,2,2], list(to.items.B))
test_eq(9, to.items.loc[to.items.TaskID==2].shape[0])

df = test_data_interpolate(index = pd.to_datetime(['2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-01:00', '2018-01-01-02:00', '2018-01-01-03:00']),)
to = TabularPandas(df,  cont_names="A", procs=Interpolate, do_setup=True)
test_eq(9, to.items.shape[0])

df = test_data_interpolate(index = [])
to = TabularPandas(df, procs=Interpolate, do_setup=True)
test_eq(0, to.items.shape[0])

<ipython-input-22-e36a25c82886>:14: UserWarning: Could not determine samples per day. Skip processing.
  warnings.warn("Could not determine samples per day. Skip processing.")


In [ ]:
#export
def _create_consistent_number_of_sampler_per_day(
    df: pd.DataFrame, n_samples_per_day: int = 24
) -> pd.DataFrame:
    """
    Remove days with less than the specified amount of samples from the DataFrame.

    Parameters
    ----------
    df : pandas.DataFrame
        the DataFrame used for the conversion.
    n_samples_per_day : integer
        the amount of samples each day in the DataFrame.

    Returns
    -------
    pandas.DataFrame
        the given DataFrame, now with a consistent amount of samples each day.
    """
    # Create a list of booleans, where each day with 'less than n_samples_per_day' samples is denoted with 'True'
    mask = df.groupby(pd.Grouper(freq="D")).apply(len)
    mask = (mask < n_samples_per_day)
    
    bad_days = list(chain.from_iterable([list(pd.date_range(start=b, periods=n_samples_per_day, freq=f'{(24 * 60) // n_samples_per_day}Min'))
                for b in mask[mask].index]))

    return df[~df.index.isin(bad_days)]

    

In [ ]:
index = pd.date_range(start='1/1/2018', periods=10, freq='3H')
df = test_data_interpolate(index)
df.tail()

,A,B,C
2018-01-01 15:00:00,4,3,7
2018-01-01 18:00:00,6,1,3
2018-01-01 21:00:00,5,8,4
2018-01-02 00:00:00,6,3,9
2018-01-02 03:00:00,2,0,4


In [ ]:
df.index.astype(int)

Int64Index([1514764800000000000, 1514775600000000000, 1514786400000000000,
            1514797200000000000, 1514808000000000000, 1514818800000000000,
            1514829600000000000, 1514840400000000000, 1514851200000000000,
            1514862000000000000],
           dtype='int64')

In [ ]:
df = _create_consistent_number_of_sampler_per_day(df, n_samples_per_day= get_samples_per_day(df))
# last two rows are removed, as they are not a "complete day"
test_eq(8, df.shape[0])

In [ ]:
#export
class FilterInconsistentSamplesPerDay(RenewablesTabularProc):
    order=100  
    include_in_new=True
    def __init__(self, group_by_col="TaskID"):
        self.group_by_col = group_by_col
        
    def setups(self, to: Tabular):
        self.n_samples_per_day = get_samples_per_day(to.items)
    
    def encodes(self, to):
        to.items = _apply_group_by(to.items, self.group_by_col, _create_consistent_number_of_sampler_per_day, 
                        n_samples_per_day=self.n_samples_per_day)
#         to.items = _create_consistent_number_of_sampler_per_day(to.items, n_samples_per_day=self.n_samples_per_day)
        
        assert (to.items.shape[0]%self.n_samples_per_day) == 0, "Incorrect number of samples after filter"

In [ ]:
test_eq(True, FilterInconsistentSamplesPerDay.order > Interpolate.order)
index = pd.date_range(start='1/1/2018', periods=30, freq='3H')
df = test_data_interpolate(list(index))
df["TaskID"] = [1 if i<11 else 2 for i in range(len(df))]
to  = TabularPandas(df, procs=FilterInconsistentSamplesPerDay, do_setup=True)
# equal to two days with eight samples per day
test_eq(16, to.items.shape[0])

In [ ]:
to.show()

""
2018-01-01 00:00:00
2018-01-01 03:00:00
2018-01-01 06:00:00
2018-01-01 09:00:00
2018-01-01 12:00:00
2018-01-01 15:00:00
2018-01-01 18:00:00
2018-01-01 21:00:00
2018-01-03 00:00:00
2018-01-03 03:00:00


In [ ]:
#export
class AddSeasonalFeatures(RenewablesTabularProc):
    order=0
    include_in_new=True
    def __init__(self, as_cont=True):
        self.as_cont = as_cont
    
    def encodes(self, to):
        as_sin = lambda value, max_value: np.sin(2*np.pi*value/max_value)
        as_cos = lambda value, max_value: np.cos(2*np.pi*value/max_value)
        
        if self.as_cont:
            to.items["MonthSin"] = as_sin(to.items.index.month, 12)
            to.items["MonthCos"] = as_cos(to.items.index.month, 12)
            to.items["DaySin"] = as_sin(to.items.index.day, 31)
            to.items["DayCos"] = as_cos(to.items.index.day, 31)
            to.items["HourSin"] = as_sin(to.items.index.hour, 24)
            to.items["HourCos"] = as_cos(to.items.index.hour, 24)
            
        else:
            to.items["Month"] = to.items.index.month
            to.items["Day"] = to.items.index.day
            to.items["Hour"] = to.items.index.hour

In [ ]:
to = get_test_data(index=['2018-01-01-01', '2018-01-01-02', '2018-01-02-03', '2018-02-01-23', '2018-02-01-13'] )
CreateTimeStampIndex("TimeStamps")(to)
AddSeasonalFeatures(as_cont=False)(to)
test_eq(np.array([1,1,1,2,2]), to.items.Month.values)
test_eq(np.array([1,1,2,1,1]), to.items.Day.values)
test_eq(np.array([1,2,3,23,13]), to.items.Hour.values)
# TODO test sin/cos

## Filter Methods

In [ ]:
#export
class FilterByCol(RenewablesTabularProc):
    "Drops rows by column."
    order = 9
    def __init__(self, col_name, drop=True, drop_col_after_filter=True):
        self.col_name = col_name
        self.drop = drop
        self.drop_col_after_filter=drop_col_after_filter

    def encodes(self, to):
        mask = to.items[self.col_name].astype(bool).values
        if self.drop: mask = ~mask
        to.items = to.items[mask]
        if self.drop_col_after_filter: to.items.drop(self.col_name, axis=1, inplace=True, errors="ignore")

In [ ]:
to = get_test_data()
to.loc[:,"C"] = [0,0,1,1,0]
FilterByCol(col_name="C", drop_col_after_filter=True, drop=True)(to)
test_eq(list(to.items.index),[0,1,4])
test_eq(to.items.columns,["A","B"])

to = get_test_data()
to.loc[:,"C"] = [0,0,1,1,0]
FilterByCol(col_name="C", drop_col_after_filter=False, drop=False)(to)
test_eq(list(to.items.index),[2,3])
test_eq(to.items.columns,["A","B", "C"])

In [ ]:
#export
class FilterYear(RenewablesTabularProc):
    "Filter a list of years. By default the years are dropped."
    order = 9
    def __init__(self, year, drop=True):
        "year(s) to filter, whether to drop or keep the years."
        year = listify(year)
        self.year = L(int(y) for y in year)
        self.drop = drop

    def encodes(self, to):
        mask = None
        for y in self.year:
            cur_mask = to.items.index.year == y
            if mask is None: mask = cur_mask
            else: mask = mask | cur_mask

        if not self.drop: mask = ~mask
        to.items.drop(to.items[mask].index, inplace=True)

In [ ]:
def test_data_filter_year():
    index = ['2018-01-01-01', '2019-01-01-02', '2020-01-02-03',] 
    return TabularPandas(pd.DataFrame(index=pd.to_datetime(index),
                      columns = ['A', 'B', 'C'] ).fillna(0))
    
to = test_data_filter_year()
FilterYear(year=2018)(to)
test_eq(np.array([2019,2020]), to.items.index.year)
to = test_data_filter_year()
FilterYear(year=2020, drop=False)(to)
test_eq(np.array([2020]), to.items.index.year)
to = test_data_filter_year()
FilterYear(year=[2018,2020], drop=True)(to)
test_eq(np.array([2019]), to.items.index.year)
to = test_data_filter_year()
FilterYear(year=[2018,2020], drop=False)(to)
test_eq(np.array([2018,2020]), to.items.index.year)

In [ ]:
#export
class FilterHalf(RenewablesTabularProc):
    "First half of the data is used for training and the other half of validation/testing."
    order = 9
    def __init__(self, drop=False, bydate=True):
        """
        Whether to drop or keep the first half.
        When bydate is true the average date, between the first and last date is used to filter the data.
        If bydate is false the amount of data is splitted by half, so that train and validation/testing have an equal amount of available data.
        """
        self.drop = drop
        self.bydate = bydate
        
    def setups(self, to: Tabular):
        df = to.items.sort_index()
        if self.bydate:
        
            self.first_date = df.index[0]
            self.last_date = df.index[-1]
            self.split_date = self.first_date + (self.last_date-self.first_date)/2
        else:
            idx = len(df)//2
            self.split_date = df.index[idx]

    def encodes(self, to):
        mask = to.items.index < self.split_date

        if not self.drop: mask = ~mask
        to.items.drop(to.items[mask].index, inplace=True)

In [ ]:
def test_data_filter_by_half(procs):
    index = ['2018-01-01-01', '2019-01-01-02', '2020-01-02-03',] 
    index=pd.to_datetime(index)
    return index, TabularPandas(pd.DataFrame(index=index,
                      columns = ['A', 'B', 'C'] ).fillna(0), procs=procs,)

index, res = test_data_filter_by_half(FilterHalf())
test_eq(index[0:2], res.items.index)
index, res = test_data_filter_by_half(FilterHalf(drop=True))
test_eq([index[2]], res.items.index)

index, res = test_data_filter_by_half(FilterHalf(bydate=False))
test_eq([index[0]], res.items.index)
index, res = test_data_filter_by_half(FilterHalf(bydate=False, drop=True))
test_eq(index[1:], res.items.index)

In [ ]:
#export
class FilterMonths(RenewablesTabularProc):
    "Filter dataframe for specific months."
    order = 9
    def __init__(self, months=range(1,13), drop=False):
        self.months = listify(months)
        self.drop = drop

    def encodes(self, to):
        mask = to.items.index.month.isin(self.months)
        if not self.drop: mask = ~mask
        to.items.drop(to.items[mask].index, inplace=True)

In [ ]:
def get_test_data_filter_month():
    to = get_test_data(index=['2018-01-01-01', '2018-02-01-02', '2018-03-02-03', '2018-04-01-23', '2018-05-01-13'])
    CreateTimeStampIndex("TimeStamps")(to)
    return to

def test_filter_month(months,drop,expected_result):
    to = get_test_data_filter_month()
    FilterMonths(months,drop)(to)
    test_eq(to.items.index.month, expected_result)
    
test_filter_month([1,2], False, [1,2])
test_filter_month(range(1,3), False, [1,2])
test_filter_month([1], False, [1])
test_filter_month([1,2], True, [3,4,5])

In [ ]:
#export
class FilterDays(RenewablesTabularProc):
    "Filter dataframe for specific months."
    order = 10
    def __init__(self, num_days):
        self.num_days = num_days
        
    def setups(self, to: Tabular):
        self.n_samples_per_day = get_samples_per_day(to.items)
        
    def encodes(self, to):
        to.items = to.items[-(self.n_samples_per_day * self.num_days):]


In [ ]:
# TODO: create test

In [ ]:
#hide
test_eq(True, FilterYear.order<FilterDays.order)
test_eq(True, FilterMonths.order<FilterDays.order)
test_eq(True, FilterByCol.order<FilterDays.order)

In [ ]:
#export
class DropCols(RenewablesTabularProc):
    "Drops rows by column name."
    include_in_new=True
    order = 10
    def __init__(self, cols):
        self.cols = listify(cols)

    def encodes(self, to):
        to.items.drop(self.cols, axis=1, inplace=True, errors="ignore")

In [ ]:
to = get_test_data()
DropCols(None)(to)
test_eq(to.items.columns, ["A", "B", "C"])
to = get_test_data()
DropCols([])(to)
test_eq(to.items.columns, ["A", "B", "C"])
to = get_test_data()
DropCols(["C"])(to)
test_eq(to.items.columns, ["A", "B"])
to = get_test_data()
DropCols(["A", "B"])(to)
test_eq(to.items.columns, ["C"])
to = get_test_data()

## Column conversion

In [ ]:
# export
class Normalize(RenewablesTabularProc):
    "Normalize per TaskId"
    order = 1
    include_in_new=True
    def __init__(self, cols_to_ignore=[]):
        self.cols_to_ignore = cols_to_ignore

    def setups(self, to: Tabular):
        self.rel_cols = [c for c in to.cont_names if c not in self.cols_to_ignore]
        self.means = getattr(to, "train", to)[self.rel_cols].mean()
        self.stds = getattr(to, "train", to)[self.rel_cols].std(ddof=0) + 1e-7

    def encodes(self, to):
        to.loc[:, self.rel_cols] = (to.loc[:, self.rel_cols] - self.means) / self.stds

    def decodes(self, to):
        to.loc[:, self.rel_cols] = to.loc[:, self.rel_cols] * self.stds + self.means

As we assume, that we have NWP as input features, we can always extract the past. 
Therefore, we should normalize before filtering the data.

In [ ]:
test_eq(True, Normalize.order<FilterMonths.order)
test_eq(True, Normalize.order<FilterByCol.order)
test_eq(True, Normalize.order<FilterYear.order)

In [ ]:
#export
class BinFeatures(TabularProc):
    "Creates bin from categorical features."
    order = 1
    include_in_new=True
    def __init__(self, column_names, bin_sizes=5):
        # TODO: Add possiblitiy to add custom bins
        self.column_names = listify(column_names)
        self.bin_sizes = listify(bin_sizes)
        if len(self.bin_sizes) == 1: self.bin_sizes = L(self.bin_sizes[0] for _ in self.column_names)

    def setups(self, to:Tabular):
        train_to = getattr(to, 'train', to)
        self.bin_edges = {c:pd.qcut(train_to.items[c], q=bs, retbins=True)[1] for c,bs in zip(self.column_names,self.bin_sizes)}


    def encodes(self, to):
        for c in self.bin_edges.keys():
            to.items.loc[:,c] = pd.cut(to.items[c], bins=self.bin_edges[c],
                                       labels=range(1, len(self.bin_edges[c])),
                                       include_lowest=True)

Test corner cases of minimum and maximum.

In [ ]:
np.random.seed(42)
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df.iloc[0,:] = 0
df.iloc[-1,:] = 100
to = TabularPandas(df, cont_names=["A", "B", "C"], y_names="D", procs=BinFeatures(column_names=["A", "B", "C"]))
test_eq(to.items.iloc[-1,:][["A", "B", "C"]].values, [5,5,5])
test_eq(to.items.iloc[0,:][["A", "B", "C"]].values, [1,1,1])
# Test for nas.
test_eq(to.items.isna().sum().sum(), 0)

Check if it is also works along with categorify.

In [ ]:
np.random.seed(42)
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df.iloc[0,:] = 0
df.iloc[-1,:] = 100
to = TabularPandas(df, cont_names=["A", "B", "C"], y_names="D", procs=[BinFeatures(column_names=["A", "B", "C"]), Categorify()])
test_eq(to.items.iloc[-1,:][["A", "B", "C"]].values, [5,5,5])
test_eq(to.items.iloc[0,:][["A", "B", "C"]].values, [1,1,1])

## Extension of TabularPandas with pre-processing

In [ ]:
from fastai.tabular.core import *

In [ ]:
#hide
#export
def _add_prop(cls, o):
    setattr(cls, camel2snake(o.__class__.__name__), o)

In [ ]:
#export
class RenewableSplits:
    pass


class ByWeeksSplitter(RenewableSplits):
    def __init__(self, every_n_weeks: int = 4):
        self.every_n_weeks = every_n_weeks
    
    @staticmethod
    def _inner(cur_dataset, every_n_weeks):
        # plus one for one week of validation data
        mask = ((cur_dataset.index.isocalendar().week % (every_n_weeks+1)) == 0).values
        indices = np.arange(len(cur_dataset))
        return list(indices[list(~mask)]), list(indices[list(mask)])
    

    def __call__(self, o):
        return self._inner(o, self.every_n_weeks)
    

class TrainTestSplitByDays(RenewableSplits):
    def __init__(self, test_size: float = 0.25):
        self.test_size = test_size
    
    @staticmethod
    def _inner(cur_dataset, test_size):
        unique_days = np.unique(cur_dataset.index.date)
        train_days, test_days = train_test_split(
            unique_days, random_state=42, test_size=0.25
        )
        train_mask = np.array([True if d in train_days else False for d in cur_dataset.index.date])
        
        indices = np.arange(len(cur_dataset))
        return list(indices[list(train_mask)]), list(indices[list(~train_mask)])
    

    def __call__(self, o):
        return self._inner(o, self.test_size)

In [ ]:
#export
class TabularRenewables(TabularPandas):
    def __init__(self, dfs, procs=None, cat_names=None, cont_names=None, do_setup=True, reduce_memory=False,
                 y_names=None, add_y_to_x=False, add_x_to_y=False, 
                 pre_process=None, device=None, splits=None, y_block=RegressionBlock(),
                 group_id="TaskID",
                inplace=False):

        self.pre_process = pre_process
        self._original_pre_process = self.pre_process
        cont_names = listify(cont_names)
        cat_names = listify(cat_names)
        y_names = listify(y_names)
        self.pre_process = listify(pre_process)
        self.group_id = group_id

        for pp in listify(procs):
            if isinstance(pp, RenewablesTabularProc):
                warnings.warn(f"Element {pp} of procs is RenewablesTabularProc, might not work with TabularPandas.")
         

        if len(self.pre_process) > 0:
            self.prepared_to = TabularPandas(dfs, y_names=y_names, 
                                             procs=self.pre_process, cont_names=cont_names,
                                          do_setup=True, reduce_memory=False, inplace=inplace, y_block=y_block)
            self.pre_process = self.prepared_to.procs
            prepared_df = self.prepared_to.items
            for pp in self.pre_process: 
                if getattr(pp, "include_in_new", False): _add_prop(self, pp)
        else:
            prepared_df = dfs

        if splits is not None: 
            if isinstance(splits, RenewableSplits): self.splits = splits(prepared_df)
            else: self.splits = splits(range_of(prepared_df))
        else:
            self.splits = None
            
        super().__init__(prepared_df,
            procs=procs,
            cat_names=cat_names,
            cont_names=cont_names,
            y_names=y_names,
            splits=self.splits,
            do_setup=do_setup,
            inplace=inplace,
            y_block=y_block,
            reduce_memory=reduce_memory)

    def new(self, df, pre_process=None, splits=None, include_preprocess=False):
        pre_process = listify(pre_process)
        if include_preprocess:
            for pp in self._original_pre_process:
                if getattr(pp, "include_in_new", False):
                    pre_process += [pp]
                    
        return type(self)(df, do_setup=False, reduce_memory=False, y_block=TransformBlock(),
                          pre_process=pre_process, splits=splits,
                          **attrdict(self, 'procs','cat_names','cont_names','y_names', 'device', 'group_id'))

    def show(self, max_n=10, **kwargs):
        to_tmp = self.new(self.all_cols[:max_n])
        to_tmp.items["TaskID"] = self.items.TaskID[:max_n]
        display_df(to_tmp.decode().items)

## Custom Dataloader

In [ ]:
#export

class ReadTabBatchRenewables(ItemTransform):
    "Transform `TabularPandas` values into a `Tensor` with the ability to decode"
    def __init__(self, to): self.to = to.new_empty()

    def encodes(self, to):
        self.task_ids = to.items[["TaskID"]]
        if not to.with_cont: res = (tensor(to.cats).long(),)
        # TODO: some pre-processing causes to.conts.values of type object, while types
        # of the dataframe are float, therefore assure conversion through astype
        # --> this is caused by Interpolate
        else: res = (tensor(to.cats).long(),tensor(to.conts.astype(float)).float())
        ys = [n for n in to.y_names if n in to.items.columns]
        # same problem as above with type of to.targ
        # preliminary bug fix 
        # is continous target?
        if getattr(to, 'regression_setup', False):
            ys_type = float
        else:
            ys_type = int
        if len(ys) == len(to.y_names): res = res + (tensor(to.targ.astype(ys_type)),)
        if to.device is not None: res = to_device(res, to.device)
        return res

    def decodes(self, o):
        o = [_maybe_expand(o_) for o_ in to_np(o) if o_.size != 0]
        vals = np.concatenate(o, axis=1)
        try: df = pd.DataFrame(vals, columns=self.to.all_col_names)
        except: df = pd.DataFrame(vals, columns=self.to.x_names)

        to = self.to.new(df)
        to.items["TaskID"]=self.task_ids.values

        return to

In [ ]:
#export
@delegates()
class TabDataLoaderRenewables(TfmdDL):
    "A transformed `DataLoader` for Tabular data"
    def __init__(self, dataset, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        if after_batch is None: after_batch = L(TransformBlock().batch_tfms)+ReadTabBatchRenewables(dataset)
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def create_batch(self, b): return self.dataset.iloc[b]
    def do_item(self, s):      return 0 if s is None else s

TabularRenewables._dl_type = TabDataLoaderRenewables

In [ ]:
#export
class NormalizePerTask(TabularProc):
    "Normalize per TaskId"
    order = 1
    include_in_new=True
    def __init__(self, task_id_col="TaskID", ignore_cont_cols=[]):
        self.task_id_col = task_id_col
        self.ignore_cont_cols = ignore_cont_cols
    def setups(self, to:Tabular):
        self.relevant_cols = L(c for c in to.cont_names if c not in self.ignore_cont_cols)
        
        self.means = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").mean()
        self.stds = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").std(ddof=0)+1e-7


    def encodes(self, to):
        to.loc[:, self.relevant_cols] = to.loc[:, self.relevant_cols].astype(np.float64)
        for task_id in to.items[self.task_id_col].unique():
            # in case this is a new task, we update the means and stds
            if task_id not in self.means.index:
                mu = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").mean()

                self.means= self.means.append(mu)
                self.stds = self.stds.append(getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").std(ddof=0)+1e-7)


            mask = to.loc[:,self.task_id_col] == task_id

            to.loc[mask, self.relevant_cols] = ((to.loc[mask, self.relevant_cols] - self.means.loc[task_id]) / self.stds.loc[task_id])

    def decodes(self, to, split_idx=None):
        for task_id in to.items[self.task_id_col].unique():
            # in case this is a new task, we update the means and stds
            if task_id not in self.means.index:
                warnings.warn("Missing task id, could not decode.")

            mask = to.loc[:,self.task_id_col] == task_id

            to.loc[mask, self.relevant_cols] = to.conts[mask] * self.stds.loc[task_id] + self.means.loc[task_id]
        return to

In [ ]:
def get_test_data_task_normalization(index=None, procs=NormalizePerTask, ignore_cols = L()):
    df = pd.DataFrame(index=range(1,11), columns = ['A', 'B', 'C'] , 
                      data=np.array([list(range(1,11)), list(range(11,21)), list(range(21,31))]).T)
    if index is not None: df["TimeStamps"] = index
    df["TaskID"] = L(1 if i <= 5  else 2 for i in range(1,11))
    index = ['2015-01-01-01', '2015-01-01-02', '2015-01-02-03', '2015-02-01-23', '2015-02-01-13',
        '2016-01-01-01', '2016-01-01-02', '2016-06-02-03', '2016-02-01-23', '2016-02-01-13'] 
    df["TimeStamps"] = index
    to = TabularRenewables(df, pre_process=CreateTimeStampIndex(col_name="TimeStamps"), 
                           procs=[NormalizePerTask(ignore_cont_cols=ignore_cols)], cont_names=["A", "B"] , y_names="C", 
                           cat_names=["TaskID"]
                          )
    df["TimeStamps"] = pd.to_datetime(index, utc=True)
    df.set_index("TimeStamps",inplace=True)
    return df,to


In [ ]:
# hide
original_df, to = get_test_data_task_normalization()
test_close(original_df.astype(float).values, to.decode().items.astype(float).values)
test_eq(np.array([[3,13],[8,18]]), to.normalize_per_task.means.values)
test_close(np.array([[1.41421366, 1.41421366],[1.41421366, 1.41421366]]), to.normalize_per_task.stds.values)
test_eq((2,2), to.normalize_per_task.means.values.shape)
test_eq((2,2), to.normalize_per_task.stds.values.shape)

In [ ]:
# hide
original_df, to = get_test_data_task_normalization(ignore_cols=L("A"))
test_close(original_df.astype(float).values, to.decode().items.astype(float).values)
test_eq((2,1), to.normalize_per_task.means.values.shape)
test_eq((2,1), to.normalize_per_task.stds.values.shape)

In [ ]:
#export
class VerifyAndNormalizeTarget(TabularProc):
    "Normalize per TaskId"
    order = 1
    include_in_new=True
    def __init__(self, reset_min_value=0.0, reset_max_value=1.05, \
                 max_value_for_normalization=1.5, task_id_col="TaskID",):
        self.task_id_col = task_id_col
        self.reset_min_value, self.reset_max_value = reset_min_value, reset_max_value
        self.max_value_for_normalization = max_value_for_normalization
    def setups(self, to:Tabular):
        self.relevant_cols = to.y_names
        
        self.maxs = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").max()
        self.mins = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").min()+1e-9


    def encodes(self, to):
#         return 
        to.loc[:, self.relevant_cols] = to.loc[:, self.relevant_cols].astype(np.float64)
        for task_id in to.items[self.task_id_col].unique():
            # in case this is a new task, we update the maxs and mins
            if task_id not in self.maxs.index:
                task_max = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").max()
                task_min = getattr(to, 'train', to)[self.relevant_cols + "TaskID"].groupby("TaskID").min()+1e-9
    
                self.maxs.append(task_max)
                self.mins.append(task_min)


            mask = to.loc[:,self.task_id_col] == task_id
            
            
            if (to.loc[mask, self.relevant_cols].max() > self.max_value_for_normalization).any():
                to.loc[mask, self.relevant_cols] = (to.loc[mask, self.relevant_cols] - self.mins.loc[task_id]) \
                                                            / (self.maxs.loc[task_id] - self.mins.loc[task_id])
                                                                                                               
            for cur_relevant_column in self.relevant_cols:
                to.loc[mask, cur_relevant_column] = to.loc[mask, cur_relevant_column].where(~(to.loc[mask, cur_relevant_column] > self.reset_max_value), 
                                                       self.reset_max_value)
                to.loc[mask, cur_relevant_column] = to.loc[mask, cur_relevant_column].where(~(to.loc[mask, cur_relevant_column] < self.reset_min_value), 
                                                       self.reset_min_value)
        return to


In [ ]:
def get_test_data_task_target_normalization(index=None, reset_min_value=0.0, reset_max_value=1.05, \
                 max_value_for_normalization=1.5):
    df = pd.DataFrame(index=range(1,11), columns = ['A', 'B', 'C'] , 
                      data=np.array([list(range(1,11)), list(range(11,21)), list(range(21,31))]).T)
    if index is not None: df["TimeStamps"] = index
    df["TaskID"] = L(1 if i <= 5  else 2 for i in range(1,11))
    index = ['2015-01-01-01', '2015-01-01-02', '2015-01-02-03', '2015-02-01-23', '2015-02-01-13',
        '2016-01-01-01', '2016-01-01-02', '2016-06-02-03', '2016-02-01-23', '2016-02-01-13'] 
    df["TimeStamps"] = index
    to = TabularRenewables(df, pre_process=CreateTimeStampIndex(col_name="TimeStamps"), 
                           procs=[VerifyAndNormalizeTarget(reset_min_value,reset_max_value,
                                                           max_value_for_normalization)], 
                           cont_names=["A"] , y_names=["B", "C"], 
                           cat_names=["TaskID"]
                          )
    df["TimeStamps"] = pd.to_datetime(index, utc=True)
    df.set_index("TimeStamps",inplace=True)
    return df,to


In [ ]:
df,to = get_test_data_task_target_normalization(reset_min_value=0, reset_max_value=1.05, 
                                                max_value_for_normalization=1.5)
# all values are larger than max_value_for_normalization, so all tasks should be normalized
test_close(0, to.ys.min())
test_close(1, to.ys.max())

df,to = get_test_data_task_target_normalization(reset_min_value=22, reset_max_value=29, 
                                                max_value_for_normalization=35)

# everything is lower than 22, so all values should be 22 for feature B
test_close(22, to.ys["B"].mean())
test_close(22, to.ys["C"].min())
test_close(29, to.ys["C"].max())

## Numpy Dataloader

In [ ]:
#export
class TabDataset(fastuple):
    "A dataset from a `TabularRenewable` object"
    # Stolen from https://muellerzr.github.io/fastblog/2020/04/22/TabularNumpy.html
    def __init__(self, to):
        self.cats = tensor(to_np(to.cats).astype(np.long))
        self.conts = tensor(to_np(to.conts).astype(np.float32))
        
        if getattr(to, 'regression_setup', False):
            ys_type = np.float32
        else:
            ys_type = np.long
        self.ys = tensor(to_np(to.ys).astype(ys_type))
        
        self.cont_names = to.cont_names
        self.cat_names = to.cat_names
        self.y_names = to.y_names

    def __getitem__(self, idx):
        idx = idx[0]
        return self.cats[idx:idx+self.bs], self.conts[idx:idx+self.bs], self.ys[idx:idx+self.bs]

    def __len__(self): return len(self.cats)
    
    def show(self, max_n=10, **kwargs): 
        df_cont = pd.DataFrame(data=self.conts[:max_n], columns=self.cont_names)
        df_cat = pd.DataFrame(data=self.cats[:max_n], columns=self.cat_names)
        df_y = pd.DataFrame(data=self.ys[:max_n], columns=self.y_names)
        display_df(pd.concat([df_cont, df_cat, df_y], axis=1))
        
    def show_batch(self, max_n=10, **kwargs): 
        self.show()


In [ ]:
#export
class TabDataLoader(DataLoader):
    def __init__(self, dataset, bs=32, num_workers=0, device='cuda', 
                 to_device=True, shuffle=False, drop_last=True,**kwargs):
        "A `DataLoader` based on a `TabDataset"
        device = device if torch.cuda.is_available() else "cpu"
        
        super().__init__(dataset, bs=bs, num_workers=num_workers, shuffle=shuffle, 
                         device=device, drop_last=drop_last, **kwargs)
        
        self.dataset.bs=bs
        if to_device:self.to_device()
    
    def create_item(self, s): return s
    
    def to_device(self, device=None):
        if device is None: device = self.device
        self.dataset.cats.to(device)
        self.dataset.conts.to(device)
        self.dataset.ys.to(device)
    
    def create_batch(self, b):
        "Create a batch of data"
        cat, cont, y = self.dataset[b]
        return cat.to(self.device), cont.to(self.device), y.to(self.device)

    def get_idxs(self):
        "Get index's to select"
        idxs = Inf.count if self.indexed else Inf.nones
        if self.n is not None: idxs = list(range(len(self.dataset)))
        return idxs

    def shuffle_fn(self):
        "Shuffle the interior dataset"
        rng = np.random.permutation(len(self.dataset))
        self.dataset.cats = self.dataset.cats[rng]
        self.dataset.conts = self.dataset.conts[rng]
        self.dataset.ys = self.dataset.ys[rng]

In [ ]:
#export
class TabDataLoaders(DataLoaders):
    def __init__(self, to, bs=64, val_bs=None, shuffle_train=True, device='cpu', **kwargs):
        train_ds = TabDataset(to.train)
        valid_ds = TabDataset(to.valid)
        val_bs = bs if val_bs is None else val_bs
        train = TabDataLoader(train_ds, bs=bs, shuffle=shuffle_train, device=device, **kwargs)
        valid = TabDataLoader(valid_ds, bs=val_bs, shuffle=False, device=device, **kwargs)
        super().__init__(train, valid, device=device, **kwargs)


# Integration Example

Lets verify that we can the split the data by weeks. 
Splitting by weeks often allows to achieve an representative validation error while reducing training effort
compared to cross validation. In this scenario four weeks are considered as training data and the next week is 
validation data, then the next four weeks are consisdered as training data, and so forth.

In [ ]:
index = pd.date_range(start='1/1/2018', end='31/12/2018', freq='6H')
df = pd.DataFrame(columns=["A", "B", "C"], index=index).fillna(1)
df.head()

,A,B,C
2018-01-01 00:00:00,1,1,1
2018-01-01 06:00:00,1,1,1
2018-01-01 12:00:00,1,1,1
2018-01-01 18:00:00,1,1,1
2018-01-02 00:00:00,1,1,1


In [ ]:
to = TabularRenewables(df, y_names="C", procs=Normalize, 
                  pre_process=AddSeasonalFeatures, 
                  splits=ByWeeksSplitter(every_n_weeks=4)
                  )

E.g. the first and fifth week is accounted as validation data.

In [ ]:
to.valid.items.index.isocalendar().week[24:30]

2018-02-04 00:00:00     5
2018-02-04 06:00:00     5
2018-02-04 12:00:00     5
2018-02-04 18:00:00     5
2018-03-05 00:00:00    10
2018-03-05 06:00:00    10
Name: week, dtype: UInt32

Lets create a dataloader and show a single batch.

In [ ]:
original_df, to = get_test_data_task_normalization()
dl = to.dataloaders(bs=4)

**normalized data:**

In [ ]:
to.items

,A,B,C,TaskID
TimeUTC,,,,
2015-01-01 01:00:00+00:00,-1.414213,-1.414213,21,1
2015-01-01 02:00:00+00:00,-0.707107,-0.707107,22,1
2015-01-02 03:00:00+00:00,0.000000,0.000000,23,1
2015-02-01 23:00:00+00:00,0.707107,0.707107,24,1
2015-02-01 13:00:00+00:00,1.414213,1.414213,25,1
2016-01-01 01:00:00+00:00,-1.414213,-1.414213,26,2
2016-01-01 02:00:00+00:00,-0.707107,-0.707107,27,2
2016-06-02 03:00:00+00:00,0.000000,0.000000,28,2
2016-02-01 23:00:00+00:00,0.707107,0.707107,29,2


**decode data (denormalized)**

In [ ]:
to.show()

,TaskID,A,B,C
TimeUTC,,,,
2015-01-01 01:00:00+00:00,1,1.0,11.0,21
2015-01-01 02:00:00+00:00,1,2.0,12.0,22
2015-01-02 03:00:00+00:00,1,3.0,13.0,23
2015-02-01 23:00:00+00:00,1,4.0,14.0,24
2015-02-01 13:00:00+00:00,1,5.0,15.0,25
2016-01-01 01:00:00+00:00,2,6.0,16.0,26
2016-01-01 02:00:00+00:00,2,7.0,17.0,27
2016-06-02 03:00:00+00:00,2,8.0,18.0,28
2016-02-01 23:00:00+00:00,2,9.0,19.0,29


**the original input dataframe:**

In [ ]:
original_df

,A,B,C,TaskID
TimeStamps,,,,
2015-01-01 01:00:00+00:00,1,11,21,1
2015-01-01 02:00:00+00:00,2,12,22,1
2015-01-02 03:00:00+00:00,3,13,23,1
2015-02-01 23:00:00+00:00,4,14,24,1
2015-02-01 13:00:00+00:00,5,15,25,1
2016-01-01 01:00:00+00:00,6,16,26,2
2016-01-01 02:00:00+00:00,7,17,27,2
2016-06-02 03:00:00+00:00,8,18,28,2
2016-02-01 23:00:00+00:00,9,19,29,2


In [ ]:
dl.one_batch()

(tensor([[2],
         [1],
         [1],
         [2]], device='cuda:0'),
 tensor([[ 0.0000,  0.0000],
         [ 1.4142,  1.4142],
         [-0.7071, -0.7071],
         [-0.7071, -0.7071]], device='cuda:0'),
 tensor([[28.],
         [25.],
         [22.],
         [27.]], device='cuda:0'))

The following gives an example on how to add a new task, that is normalized based on the first year. E.g. when the features are numerical weather predictions. As those are themselves forecasts, we can always extract the past and use the data for standardization.

In [ ]:
original_df, to = get_test_data_task_normalization()
original_df["TaskID"] = 3

In [ ]:
pd.options.mode.chained_assignment=None
# setups task normalization
to_new = to.new(original_df, pre_process=FilterYear(2016, drop=True))
to_new.process()
to_new.items

,A,B,C,TaskID
TimeStamps,,,,
2015-01-01 01:00:00+00:00,-1.414213,-1.414213,21,3
2015-01-01 02:00:00+00:00,-0.707107,-0.707107,22,3
2015-01-02 03:00:00+00:00,0.000000,0.000000,23,3
2015-02-01 23:00:00+00:00,0.707107,0.707107,24,3
2015-02-01 13:00:00+00:00,1.414213,1.414213,25,3


In [ ]:
to_new.items.describe()

,A,B,C,TaskID
count,5.000000e+00,5.000000e+00,5.000000,5.0
mean,-4.440892e-17,-4.440892e-17,23.000000,3.0
std,1.118034e+00,1.118034e+00,1.581139,0.0
min,-1.414213e+00,-1.414213e+00,21.000000,3.0
25%,-7.071067e-01,-7.071067e-01,22.000000,3.0
50%,0.000000e+00,0.000000e+00,23.000000,3.0
75%,7.071067e-01,7.071067e-01,24.000000,3.0
max,1.414213e+00,1.414213e+00,25.000000,3.0


Normalize the second year based on the mean and std of the first year.

In [ ]:
to_new = to.new(original_df, pre_process=FilterYear(2016, drop=False))
to_new.process()

As the data has larger values in the second year, the normalization is quite off.

In [ ]:
to_new.items

,A,B,C,TaskID
TimeStamps,,,,
2016-01-01 01:00:00+00:00,2.121320,2.121320,26,3
2016-01-01 02:00:00+00:00,2.828427,2.828427,27,3
2016-06-02 03:00:00+00:00,3.535534,3.535534,28,3
2016-02-01 23:00:00+00:00,4.242640,4.242640,29,3
2016-02-01 13:00:00+00:00,4.949747,4.949747,30,3


This can also be seen in the summary:

In [ ]:
to_new.items.describe()

,A,B,C,TaskID
count,5.000000,5.000000,5.000000,5.0
mean,3.535534,3.535534,28.000000,3.0
std,1.118034,1.118034,1.581139,0.0
min,2.121320,2.121320,26.000000,3.0
25%,2.828427,2.828427,27.000000,3.0
50%,3.535534,3.535534,28.000000,3.0
75%,4.242640,4.242640,29.000000,3.0
max,4.949747,4.949747,30.000000,3.0


Let's check a dataloader and verify if we can display the data.

In [ ]:
to_new.dataloaders(bs=4).show_batch()

,TaskID,A,B,C
0,3,10.0,20.0,30.0
1,3,9.0,19.0,29.0
2,3,6.0,16.0,26.0
3,3,8.0,18.0,28.0
